In [6]:
import librosa 
import librosa.display
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from tensorflow import keras 
from tensorflow.keras.preprocessing import image_dataset_from_directory, image 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications import VGG19, ResNet50
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, InputLayer, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight



In [2]:
path = Path.cwd()/'data'
answers = pd.read_csv('answers.csv')

In [ ]:
def import_data(which_path=2):
    if which_path==1:
        images_directory = path/'train_images'
    else: 
        images_directory = path/'train_images2'
    cb_training = image_dataset_from_directory(images_directory, labels='inferred', image_size=(97,97), subset='training', validation_split=.2, seed=10)
    cb_validation = image_dataset_from_directory(images_directory, labels='inferred', image_size=(97,97), subset='validation', validation_split=.2, seed=10)
    return (cb_training, cb_validation)


def base_set_up(filename, patience=2):
    
    xx =compute_class_weight(class_weight='balanced',classes=np.unique(answers.label), y=answers.label)
    class_weight = dict(zip(np.unique(answers.label), xx))
    METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]

    earlystop = EarlyStopping(monitor='val_accuracy',verbose=1, patience=patience)
    checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', save_best_only=True, save_weights_only=False, verbose=1)
    return (class_weight, METRICS, earlystop, checkpoint)

# Continue Training Resnet50 w/ cb validation 

In [ ]:
# Early stopping 

# model = None
# keras.backend.clear_session()

filename = 'resnet-cb.h5'
cb_training, cb_validation  = import_data()
class_weight, METRICS, earlystop, checkpoint = base_set_up(filename,patience=3)

# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(97,97,3))
# for layer in base_model.layers:
#     layer.trainable = False

# x = base_model.output
# x = Flatten()(x)

# x = Dense(500, activation='relu')(x)
# x = Dropout(.5)(x)
# x = Dense(500, activation='relu')(x)
# x = Dropout(.5)(x)


# predictions = Dense(1, activation='sigmoid')(x)
# optimizer = Adam(learning_rate=0.00001)

# # model =  Model(inputs=base_model.input, outputs=predictions)
# # model.compile(optimizer=optimizer,loss='binary_crossentropy', metrics=METRICS)


model = load_model(filename)
history = model.fit(cb_training, epochs=10,validation_data=cb_validation, class_weight = class_weight, callbacks=[earlystop, checkpoint])

Found 30000 files belonging to 2 classes.
Using 24000 files for training.
Found 30000 files belonging to 2 classes.
Using 6000 files for validation.
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/10


2022-03-22 16:12:36.399320: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


750/750 [==============================] - 644s 853ms/step - loss: 0.2445 - tp: 5225.0000 - fp: 2453.0000 - tn: 15919.0000 - fn: 403.0000 - accuracy: 0.8810 - precision: 0.6805 - recall: 0.9284 - auc: 0.9612 - prc: 0.8759 - val_loss: 0.2621 - val_tp: 1259.0000 - val_fp: 564.0000 - val_tn: 4037.0000 - val_fn: 140.0000 - val_accuracy: 0.8827 - val_precision: 0.6906 - val_recall: 0.8999 - val_auc: 0.9556 - val_prc: 0.8623

Epoch 00001: val_accuracy improved from -inf to 0.88267, saving model to resnet-cb.h5


/opt/anaconda3/envs/metis/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
750/750 [==============================] - 553s 737ms/step - loss: 0.2388 - tp: 5195.0000 - fp: 2349.0000 - tn: 16023.0000 - fn: 433.0000 - accuracy: 0.8841 - precision: 0.6886 - recall: 0.9231 - auc: 0.9633 - prc: 0.8822 - val_loss: 0.2453 - val_tp: 1229.0000 - val_fp: 480.0000 - val_tn: 4121.0000 - val_fn: 170.0000 - val_accuracy: 0.8917 - val_precision: 0.7191 - val_recall: 0.8785 - val_auc: 0.9552 - val_prc: 0.8609

Epoch 00002: val_accuracy improved from 0.88267 to 0.89167, saving model to resnet-cb.h5
Epoch 3/10
750/750 [==============================] - 474s 632ms/step - loss: 0.2265 - tp: 5245.0000 - fp: 2248.0000 - tn: 16124.0000 - fn: 383.0000 - accuracy: 0.8904 - precision: 0.7000 - recall: 0.9319 - auc: 0.9667 - prc: 0.8950 - val_loss: 0.2353 - val_tp: 1162.0000 - val_fp: 385.0000 - val_tn: 4216.0000 - val_fn: 237.0000 - val_accuracy: 0.8963 - val_precision: 0.7511 - val_recall: 0.8306 - val_auc: 0.9546 - val_prc: 0.8623

Epoch 00003: val_accuracy improved from 0